# LAB 2 WEEK PRESENTATION
## DATA CLEANING
### Costanza and Tommaso 

In [269]:
### Import libraries
import pandas as pd
import numpy as np

#### Aggregate data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them.


In [270]:
file1 = pd.read_csv("file1.csv")
file2 = pd.read_csv("file2.csv")
file3 = pd.read_csv("file3.csv")

dfList = [file1, file2, file3]

def lower_case_column_names(df):
    df.columns=[i.lower() for i in df.columns]

list(map(lower_case_column_names, dfList))

[None, None, None]

#### Standardizing header names and Aggregate data into one Data Frame using Pandas.

In [271]:
file1.rename(columns = {"st": "state"}, inplace=True)
file2.rename(columns = {"st": "state"}, inplace=True)
df = pd.concat([file1,file2,file3], axis=0)

#### Deleting and rearranging columns: delete the column customer as it is only a unique identifier for each row of data

In [272]:
df.reset_index(drop=True, inplace=True)
df = df.drop(labels=['customer'], axis=1)
df.head(10)

,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,Arizona,F,Bachelor,697953.59%,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,Nevada,F,Bachelor,1288743.17%,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,California,M,Bachelor,764586.18%,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,Washington,M,High School or Below,536307.65%,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
5,Oregon,F,Bachelor,825629.78%,62902.0,69.0,1/0/00,Personal Auto,Two-Door Car,159.383042
6,Oregon,F,College,538089.86%,55350.0,67.0,1/0/00,Corporate Auto,Four-Door Car,321.600000
7,Arizona,M,Master,721610.03%,0.0,101.0,1/0/00,Corporate Auto,Four-Door Car,363.029680
8,Oregon,M,Bachelor,2412750.40%,14072.0,71.0,1/0/00,Corporate Auto,Four-Door Car,511.200000
9,Oregon,F,College,738817.81%,28812.0,93.0,1/0/00,Special Auto,Four-Door Car,425.527834


#### Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints )

In [273]:
# Drop "%" from the variable
dropPerc = lambda x : x.replace ("%", "") if type(x) == str else x
df["customer lifetime value"] = df["customer lifetime value"].apply(dropPerc)
print(df['customer lifetime value'].isnull().sum())  ## check how many missing left

2944


####  Replacing null values – Replace missing values with means of the column (for numerical columns)

In [274]:
## check if they are numeric
from pandas.api.types import is_numeric_dtype
for i in df[['income', 'monthly premium auto', 'total claim amount','customer lifetime value']]: 
    print(is_numeric_dtype(i))

False
False
False
False


In [275]:
## function to fill null with mean 
def fillMean(col): 
    col = col.astype(float)
    col.fillna(col.mean(), inplace=True)

In [276]:
fillMean(df['monthly premium auto'])
fillMean(df['total claim amount'])
fillMean(df['customer lifetime value'])

In [277]:
## Income (transform nan in 0 first)
print(df.income.head(3))
df['income'].replace(0, np.nan, inplace=True)
df['income'].fillna(df['income'].mean(), inplace=True)
df['income'] = df['income'].round().astype(int)

'''
for i in df.columns:
    if is_numeric_dtype(df[i]) == True:
        df[i].fillna(df[i].mean(), inplace=True)
'''

df.income.head(3)

0        0.0
1        0.0
2    48767.0
Name: income, dtype: float64


0    50510
1    50510
2    48767
Name: income, dtype: int32

#### Clean the number of open complaints and extract the middle number which is changing between records. Pay attention that the number of open complaints is a categorical feature

In [278]:
# Transform to string and then keep only the 2nd digit (middle number) when more than 1
print(df["number of open complaints"])
df["number of open complaints"] = df["number of open complaints"].astype(str)

myF = lambda x: x[2] if x.count('') > 4 else x
df["number of open complaints2"] = df["number of open complaints"].apply(myF)
print(df['number of open complaints'].isnull().sum())

# check if correct
df["number of open complaints2"].unique()

0        1/0/00
1        1/0/00
2        1/0/00
3        1/0/00
4        1/0/00
          ...  
12069         0
12070         0
12071         3
12072         0
12073         0
Name: number of open complaints, Length: 12074, dtype: object
0


array(['0', '2', '1', '3', '5', '4', 'nan'], dtype=object)

#### Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns

In [279]:
# clean gender and check
df['gender'].replace('Femal', 'F', inplace=True)
df['gender'].replace('female', 'F', inplace=True)
df['gender'].replace('Male', 'M', inplace=True)
print(df.gender.unique())

# clean state and check
df.state = list(map(lambda x: "Arizona" if x == "AZ" else x, df.state))
df.state = list(map(lambda x: "Washington" if x == "WA" else x, df.state))
print(df.state.unique())

[nan 'F' 'M']
['Washington' 'Arizona' 'Nevada' 'California' 'Oregon' 'Cali' nan]


#### Fill nan values of Custome life value with the mean of the column

In [280]:
# Bucketing the data -
# Write a function to replace column "State" to different zones.
# California as West Region, Oregon as North West, and Washington as East,
# and Arizona and Nevada as Cent

df['region'] = ['W' if x=='California' \
                else 'NW' if x=='Oregon' \
                else 'E' if x=='Washington'\
                else 'C' if (x=='Arizona' or x=='Nevada')\
                else 'Nan' for x in df['state']]

df['region'].unique()

array(['E', 'C', 'W', 'NW', 'Nan'], dtype=object)

In [281]:
#%% with function 
def regionAssignment(state):
    state.replace("California", "WE", inplace=True)
    state.replace("Oregon", "NE", inplace=True)
    state.replace("Washington", "E", inplace=True)
    state.replace("Arizona", "C", inplace=True)
    state.replace("Nevada", "C", inplace=True)
    state.replace("Cali", "NaN", inplace=True)
    
regionAssignment(df['state'])
print(df['state'].unique())

['E' 'C' 'WE' 'NE' 'NaN' nan]


#### Standardizing the data – Use string functions to standardize the text data (lower case)

In [282]:
for i in df.columns:
    if is_numeric_dtype(df[i]) == False:
        df[i] = df[i].str.lower()
df.head(3)


d
